# Part 1: All Samples 

## 1. Deep Neural Network
###### All rows with missing values or 0s are removed. 891 observations

In [ ]:
# Import necessary modules
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, RepeatVector, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import MIDASpy as md

import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'serif' 
# you can change the font to other fonts found from your Word document.
plt.rcParams['font.size'] = 10 

In [ ]:
df_raw = pd.read_csv('/home/Data_Ubuntu/E1_891.csv')

In [ ]:
df_raw.shape

In [ ]:
cat_vars = ['carriercode', 'year', 'quarter']

In [ ]:
cont_vars = ['OPOR', 'lyield', 'lLFP', 'lfleetutil', 'lfueleff', 'lpOntime',
       'lmisbagg', 'ltotaldelay', 'lcomplaint', 'lhetero', 'lavglandfee',
       'lsparsity', 'lMKTshare', 'ltdomt_cost', 'lrevpaxenplaned', 'lempfte', 'occasion']

In [ ]:
var_list = [df_raw[cont_vars]]

In [ ]:
data_cat, cat_var_list = md.cat_conv(df_raw[cat_vars])

In [ ]:
var_list.append(data_cat)

In [ ]:
df = pd.concat(var_list, axis=1)

In [ ]:
df.head()

In [ ]:
target = np.array(df[['OPOR']])

In [ ]:
predictors = np.array(df.drop('OPOR',axis=1))

In [ ]:
keras.utils.set_random_seed(1234) # 0.0120 loss for this random seed
# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Specify the model_dnn_test
layer = Dense(11, )

model_dnn = Sequential()
model_dnn.add(Dense(500, activation='elu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(450, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(400, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(600, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(350, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(300, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(250, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(200, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(100, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())

model_dnn.add(Dense(50, activation='elu',  kernel_initializer=keras.initializers.HeNormal()))
model_dnn.add(BatchNormalization())
model_dnn.add(Dropout(0.2)) # drop out

model_dnn.add(Dense(1))

# Compile the model_dnn_test
model_dnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience = 2)

# Fit the model_dnn_test
model_dnn.fit(predictors, target, epochs = 100, validation_split = 0.1)
          #callbacks = [early_stopping_monitor])


In [ ]:
pred_opor_dnn = pd.DataFrame(model_dnn.predict(predictors))
pred_opor_dnn.columns = ['OPOR_pred_dnn']

In [ ]:
mse = mean_squared_error(target, model_dnn.predict(predictors))
r2 = r2_score(target, model_dnn.predict(predictors))

print("Mean Squared Error DNN:", mse)
print("R-squared Score DNN:", r2)

In [ ]:
df_dnn = pd.concat([df, pred_opor_dnn, df_raw['carriercode']], axis =1)
# df_dnn.to_csv('df_cnn_6.9.csv')

In [ ]:
plt.plot(df_dnn['OPOR'], label= "Actual OPOR", color = 'black')
plt.plot(df_dnn['OPOR_pred_dnn'], label= "DNN Predicted OPOR", color = 'red')
plt.xlabel('No. of Observations')
plt.ylabel('OPOR')
plt.xticks([])

plt.title('All Carriers All Observations 2004Q1 to 2019Q4')
plt.legend()

## 2. XGB

In [ ]:
#Creating an XGBoost regressor
model_xgb = xgb.XGBRegressor()

#Training the model on the training data
model_xgb.fit(predictors, target)

#Making predictions on the test set
pred_xgb = model_xgb.predict(predictors)

# Calculate the mean squared error and R-squared score
mse_xgb = mean_squared_error(target, pred_xgb)
r2_xgb = r2_score(target, pred_xgb)

print("Mean Squared Error XGB:", round(mse_xgb, 10))
print("R-squared Score XGB:", r2_xgb)

In [ ]:
pred_xgb = pd.DataFrame(model_xgb.predict(predictors))
pred_xgb.columns = ['OPOR_pred_xgb']

In [ ]:
df_xgb = pd.concat([df_dnn, pred_xgb], axis =1)


In [ ]:
df_xgb.to_csv('df_xgb_all.csv')

In [ ]:
df_xgb['dnn_dif2'] = (df_xgb.OPOR - df_xgb.OPOR_pred_dnn)*(df_xgb.OPOR - df_xgb.OPOR_pred_dnn)
df_xgb['xgb_dif2'] = (df_xgb.OPOR - df_xgb.OPOR_pred_xgb)*(df_xgb.OPOR - df_xgb.OPOR_pred_xgb)

# find the mean of dnn_dif2 and xgb_dif2 group by carriercode
df_xgb.groupby('carriercode')['dnn_dif2'].mean()
df_xgb.groupby('carriercode')['xgb_dif2'].mean()


In [ ]:
df_xgb.shape

### Read in Statafile

In [ ]:
# read in Statafile
stata_all = pd.read_csv('/home/Data_Ubuntu/StataOut_all.csv')

In [ ]:
stata_all.columns

In [ ]:
plt.plot(df_xgb['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(df_xgb['OPOR_pred_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(df_xgb['OPOR_pred_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(stata_all['pout_fe_all'], label= "Fixed Effects Predicted OPOR", color = 'Orange', linestyle ='dashed')
plt.plot(stata_all['pout_xtmixed_all'], label= "Mixed Effects Predicted OPOR", color = 'Purple', linestyle ='dashdot')

plt.xlabel('No. of Observations')
plt.ylabel('OPOR')
plt.xticks([])

plt.title('All Carriers All Observations 2004Q1 to 2019Q4')
plt.legend()
plt.legend(loc=4, prop={'size': 8})

plt.savefig('AllCarrierCombined.pdf')

In [ ]:
xgb_american = df_xgb[df_xgb['carriercode'] == 6]
stata_american = stata_all[stata_all['carriercode'] == 6]

xgb_delta = df_xgb[df_xgb['carriercode'] == 11]
stata_delta = stata_all[stata_all['carriercode'] == 11]

xgb_sw = df_xgb[df_xgb['carriercode'] == 24]
stata_sw = stata_all[stata_all['carriercode'] == 24]

xgb_united = df_xgb[df_xgb['carriercode'] == 26]
stata_united = stata_all[stata_all['carriercode'] == 26]

plt.figure(figsize = (12,8))

## American 
plt.subplot(2,2,1)
plt.plot(xgb_american['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(xgb_american['OPOR_pred_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(xgb_american['OPOR_pred_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(stata_american['pout_fe_all'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(stata_american['pout_xtmixed_all'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Quarters from 2004Q1 to 2019Q4')
plt.ylabel('OPOR')
plt.title('American Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})


## Delta
plt.subplot(2,2,2)
plt.plot(xgb_delta['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(xgb_delta['OPOR_pred_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(xgb_delta['OPOR_pred_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(stata_delta['pout_fe_all'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(stata_delta['pout_xtmixed_all'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Quarters from 2004Q1 to 2019Q4')
plt.ylabel('OPOR')
plt.title('Delta Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## Southwest
plt.subplot(2,2,3)
plt.plot(xgb_sw['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(xgb_sw['OPOR_pred_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(xgb_sw['OPOR_pred_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(stata_sw['pout_fe_all'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(stata_sw['pout_xtmixed_all'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Quarters from 2004Q1 to 2019Q4')
plt.ylabel('OPOR')
plt.title('SouthWest Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## United
plt.subplot(2,2,4)
plt.plot(xgb_united['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(xgb_united['OPOR_pred_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(xgb_united['OPOR_pred_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(stata_united['pout_fe_all'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(stata_united['pout_xtmixed_all'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Quarters from 2004Q1 to 2019Q4')
plt.ylabel('OPOR')
plt.title('United Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

plt.savefig('Big4All.pdf')

### XGB Feature Importance

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
#Creating an XGBoost regressor
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
#Training the model on the training data
model_xgb.fit(predictors, target)
permutation_importance(model_xgb.fit, random_state=1)


In [ ]:
perm_xgb = PermutationImportance(xgb_best_model, random_state=1).fit(train_x, train_y)

df_results = pd.DataFrame(data=perm_train_xgb.results_, columns= feature_for_plot)
feat_imps = df_results.mean().sort_values(ascending=False)
df_results = df_results[feat_imps.index]
fig = px.box(df_results.melt(), x='variable', y='value', orientation='v', width=700, height=400, 
            title = 'Permutation Feature Importance - Train Data -XGB')
fig.add_trace(go.Scatter(x=feat_imps.index, y=feat_imps.values, mode='markers', marker=dict(color='red'), name = 'Mean'))
fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="green",
    plot_bgcolor='white'
    )


fig.update_xaxes(showline=True, linewidth=1, linecolor='black',title_font_family="Helvetica")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.85
))

# plotly.offline.plot(fig, filename='fi_test.html')

fig

In [ ]:
df.columns

### XGB Partical Dependence Plot

In [ ]:
features = ['lyield', 'lLFP', 'lfleetutil', 'lfueleff', 'lpOntime',
       'lmisbagg', 'ltotaldelay', 'lcomplaint', 'lhetero', 'lavglandfee',
       'lsparsity', 'lMKTshare', 'ltdomt_cost', 'lrevpaxenplaned', 'lempfte',
       'occasion', 'carriercode_1', 'carriercode_2', 'carriercode_3',
       'carriercode_5', 'carriercode_6', 'carriercode_7', 'carriercode_8',
       'carriercode_9', 'carriercode_10', 'carriercode_11', 'carriercode_13',
       'carriercode_14', 'carriercode_15', 'carriercode_16', 'carriercode_17',
       'carriercode_18', 'carriercode_19', 'carriercode_20', 'carriercode_23',
       'carriercode_24', 'carriercode_25', 'carriercode_26', 'carriercode_27',
       'carriercode_28', 'year_2004', 'year_2005', 'year_2006', 'year_2007',
       'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012',
       'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017',
       'year_2018', 'year_2019', 'quarter_1', 'quarter_2', 'quarter_3',
       'quarter_4']


In [ ]:
predictors_forxgb = df.drop('OPOR',axis=1)

In [ ]:
predictors_forxgb

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
model_xgb1 = xgb.XGBRegressor()
model_xgbfit1 = model_xgb.fit(predictors, target)


In [ ]:
xgb_disp = PartialDependenceDisplay.from_estimator(model_xgbfit1, predictors_forxgb, features, n_jobs=3, grid_resolution=20)

In [ ]:
xgb_disp.plot(line_kw={"label": "XGB", "color": "#dc5c04"})

xgb_disp.figure_.set_size_inches(12, 40)
xgb_disp.axes_[0, 0].legend()
xgb_disp.axes_[0, 1].legend()

# Robustness Test on DNN

#### change to RELU; 0.0203 Loss

In [ ]:
keras.utils.set_random_seed(1234) 
# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Specify the model1
layer = Dense(11, )

model1 = Sequential()
model1.add(Dense(1000, activation='relu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(800, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(700, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(600, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(500, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(400, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(300, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(200, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(100, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())

model1.add(Dense(50, activation='relu',  kernel_initializer=keras.initializers.HeNormal()))
model1.add(BatchNormalization())
model1.add(Dropout(0.15)) # drop out

model1.add(Dense(1))

# Compile the model1
model1.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience = 2)

# Fit the model1
model1fit = model1.fit(predictors, target, epochs = 50, validation_split = 0.1)
          #callbacks = [early_stopping_monitor])



#### 0.5 drop out: 0.0178

In [ ]:
keras.utils.set_random_seed(1234) 
# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Specify the model2
layer = Dense(11, )

model2 = Sequential()
model2.add(Dense(1000, activation='elu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(800, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(700, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(600, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(500, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(400, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(300, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(200, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(100, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())

model2.add(Dense(50, activation='relu',  kernel_initializer=keras.initializers.HeNormal()))
model2.add(BatchNormalization())
model2.add(Dropout(0.5)) # drop out

model2.add(Dense(1))

# Compile the model2
model2.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience = 2)

# Fit the model2
model2fit = model2.fit(predictors, target, epochs = 50, validation_split = 0.1)
          #callbacks = [early_stopping_monitor])


#### 15 layer 0.5 drop out: 0.0262 loss

In [ ]:
keras.utils.set_random_seed(1234) 
# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Specify the model3
layer = Dense(16, )

model3 = Sequential()
model3.add(Dense(1000, activation='relu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(900, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(800, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(700, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(600, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(500, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(400, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(300, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(200, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(150, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(100, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(50, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(40, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(30, activation='relu', kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())

model3.add(Dense(20, activation='relu',  kernel_initializer=keras.initializers.HeNormal()))
model3.add(BatchNormalization())
model3.add(Dropout(0.5)) # drop out

model3.add(Dense(1))

# Compile the model3
model3.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience = 2)

# Fit the model3
model3fit = model3.fit(predictors, target, epochs = 50, validation_split = 0.1)
          #callbacks = [early_stopping_monitor])


#### 11 layers, 0.5 drop out, validation 0.2: loss 0.0219

In [ ]:
keras.utils.set_random_seed(1234) 
# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Specify the model4
layer = Dense(11, )

model4 = Sequential()
model4.add(Dense(1000, activation='elu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(800, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(700, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(600, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(500, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(400, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(300, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(200, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(100, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())

model4.add(Dense(50, activation='relu',  kernel_initializer=keras.initializers.HeNormal()))
model4.add(BatchNormalization())
model4.add(Dropout(0.5)) # drop out

model4.add(Dense(1))

# Compile the model4
model4.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
# early_stopping_monitor = EarlyStopping(patience = 5)

# Fit the model4
model4fit= model4.fit(predictors, target, epochs = 50, validation_split = 0.2)
          # callbacks = [early_stopping_monitor])


#### Original Model 0.0162 Loss

In [ ]:
keras.utils.set_random_seed(1234) # 0.0162 loss for this random seed

# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Specify the model
layer = Dense(11, ) # 11 layers

model = Sequential()
model.add(Dense(1000, activation='elu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(800, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(700, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(600, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(500, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(400, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(300, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(200, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(100, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())

model.add(Dense(50, activation='elu',  kernel_initializer=keras.initializers.HeNormal()))
model.add(BatchNormalization())
model.add(Dropout(0.15)) # drop out

model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience = 2)

# Fit the model
modelfit = model.fit(predictors, target, epochs = 50, validation_split = 0.1)
          #callbacks = [early_stopping_monitor])

#### Plot

In [ ]:
# Create the plot
plt.plot(modelfit.history['val_loss'], 'red', label = 'Usded Model')
plt.plot(model1fit.history['val_loss'], 'blue', label = 'Model 1')
plt.plot(model2fit.history['val_loss'], 'orange', label = 'Model 2')
plt.plot(model3fit.history['val_loss'], 'green', label = 'Model 3')
plt.plot(model4fit.history['val_loss'], 'purple', label = 'Model 4')

plt.xlabel('Epochs')
plt.ylabel('Validation score')
plt.title('Comparing Differnt DNN Models')
plt.legend()

plt.savefig('ComparingDNNModel.pdf')

# PART 2 Monte Carlo Cross Validation 80/20 Split

In [ ]:
df_raw = pd.read_csv('/home/Data_Ubuntu/E1_891.csv')

In [ ]:
df_raw = df_raw[df_raw.carriercode != 3]
df_raw = df_raw[df_raw.carriercode != 5]
df_raw = df_raw[df_raw.carriercode != 7]
df_raw = df_raw[df_raw.carriercode != 17]

In [ ]:
df_raw.carriercode.unique()
df = df_raw

In [ ]:
# convert catigorical var
cat_vars = ['carriercode', 'year', 'quarter']
# cat_vars = ['quarter']
data_cat, cat_var_list = md.cat_conv(df[cat_vars])

# define continuous variables
cont_vars = ['OPOR', 'lyield', 'lLFP', 'lfleetutil', 'lfueleff', 'lpOntime',
       'lmisbagg', 'ltotaldelay', 'lcomplaint', 'lhetero', 'lavglandfee',
       'lsparsity', 'lMKTshare', 'ltdomt_cost', 'lrevpaxenplaned', 'lempfte', 'occasion', 'carriercode']

# combine data
total_list = [df[cont_vars]]
total_list.append(data_cat)

total_df = pd.concat(total_list, axis=1)

In [ ]:
# randomly pick 80% of the data as training, 20% as testing, stratified by carriercode
from sklearn.model_selection import train_test_split
train_cv, test_cv = train_test_split(total_df, test_size=0.2, stratify=total_df['carriercode'])

In [ ]:
y_train_cv = np.array(train_cv[['OPOR']])
X_train_cv = np.array(train_cv.drop('OPOR',axis=1))

y_test_cv = np.array(test_cv[['OPOR']])
X_test_cv = np.array(test_cv.drop('OPOR',axis=1))

In [ ]:
# print unique carriercode from training and testing and sort them
print(sorted(train_cv.carriercode.unique()))
print(sorted(test_cv.carriercode.unique()))

In [ ]:
y_train_cv.shape

In [ ]:
train_cv.to_csv('train_cv.csv', index = False)
test_cv.to_csv('test_cv.csv', index = False)

In [ ]:
train_cv.shape

In [ ]:
test_cv.shape

## DNN

In [ ]:
keras.utils.set_random_seed(1234) # 0.0053 loss for this random seed
# Save the number of columns in predictors: n_cols
n_cols = X_train_cv.shape[1]

# Specify the model_dnn_test
layer = Dense(11, )

model_dnn_cv = Sequential()

model_dnn_cv.add(Dense(500, activation='elu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(450, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(400, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(600, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(350, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(300, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(250, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(200, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(100, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())

model_dnn_cv.add(Dense(50, activation='elu',  kernel_initializer=keras.initializers.HeNormal()))
model_dnn_cv.add(BatchNormalization())
model_dnn_cv.add(Dropout(0.2)) # drop out

model_dnn_cv.add(Dense(1))

# Compile the model_dnn_cv
model_dnn_cv.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience = 2)

# Fit the model_dnn_cv
model_dnn_cv.fit(X_train_cv, y_train_cv, epochs = 100, validation_split = 0.1)
          #callbacks = [early_stopping_monitor])


#### DNN Prediction

In [ ]:
pred_train_dnn_cv = pd.DataFrame(model_dnn_cv.predict(X_train_cv))
pred_train_dnn_cv.columns = ['OPOR_pred_train_dnn_cv']
# pred_test.to_csv('pred_test1.csv')

In [ ]:
mse_dnn_train_cv = mean_squared_error(y_train_cv, model_dnn_cv.predict(X_train_cv))
r2_dnn_train_cv = r2_score(y_train_cv, model_dnn_cv.predict(X_train_cv))

print("Mean Squared Error DNN Train CV:",mse_dnn_train_cv)
print("R-squared Score DNN Train CV:", r2_dnn_train_cv)

In [ ]:
pred_test_dnn_cv = pd.DataFrame(model_dnn_cv.predict(X_test_cv))
pred_test_dnn_cv.columns = ['OPOR_pred_test_dnn_cv']
# pred_test.to_csv('pred_test1.csv')

In [ ]:
mse_dnn_test_cv = mean_squared_error(y_test_cv, model_dnn_cv.predict(X_test_cv))
r2_dnn_test_cv = r2_score(y_test_cv, model_dnn_cv.predict(X_test_cv))

print("Mean Squared Error DNN Test CV:",mse_dnn_test_cv)
print("R-squared Score DNN Test CV:", r2_dnn_test_cv)

In [ ]:
df_dnn_train_cv = pd.concat((train_cv[['OPOR', 'carriercode', 'occasion']].reset_index(), pred_train_dnn_cv), axis =1)
#df_dnn_test.to_csv('df_dnn_test.csv')

In [ ]:
df_dnn_test_cv = pd.concat((test_cv[['OPOR', 'carriercode', 'occasion']].reset_index(), pred_test_dnn_cv), axis =1)
#df_dnn_test.to_csv('df_dnn_test.csv')

## XGB

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#Creating an XGBoost regressor
model_xgb_cv = xgb.XGBRegressor()

#Training the model on the training data
model_xgb_cv.fit(X_train_cv, y_train_cv)

#Making predictions on the train data
pred_xgb_train_cv = model_xgb_cv.predict(X_train_cv)

#Making predictions on the test set
pred_xgb_test_cv = model_xgb_cv.predict(X_test_cv)

# Calculate the mean squared error and R-squared score for Train
mse_xgb_train_cv = mean_squared_error(y_train_cv, pred_xgb_train_cv)
r2_xgb_train_cv  = r2_score(y_train_cv, pred_xgb_train_cv)

# Calculate the mean squared error and R-squared score
mse_xgb_test_cv  = mean_squared_error(y_test_cv, pred_xgb_test_cv)
r2_xgb_test_cv  = r2_score(y_test_cv, pred_xgb_test_cv)

print("Mean Squared Error XGB Train CV:", mse_xgb_train_cv)
print("R-squared Score XGB Train CV:", r2_xgb_train_cv)

print("Mean Squared Error XGB Test CV:", mse_xgb_test_cv)
print("R-squared Score XGB Test CV:", r2_xgb_test_cv)

In [ ]:
pred_xgb_train_cv = pd.DataFrame(model_xgb_cv.predict(X_train_cv))

pred_xgb_train_cv.columns = ['OPOR_pred_train_xgb_cv']

In [ ]:
pred_xgb_test_cv = pd.DataFrame(model_xgb_cv.predict(X_test_cv))

pred_xgb_test_cv.columns = ['OPOR_pred_test_xgb_cv']

In [ ]:
df_xgb_train_cv = pd.concat([df_dnn_train_cv, pred_xgb_train_cv], axis =1)

In [ ]:
df_xgb_test_cv = pd.concat([df_dnn_test_cv, pred_xgb_test_cv], axis =1)

In [ ]:
df_xgb_train_cv.columns

In [ ]:
df_xgb_train_cv['dnn_dif2'] = (df_xgb_train_cv.OPOR - df_xgb_train_cv.OPOR_pred_train_dnn_cv)*(df_xgb_train_cv.OPOR - df_xgb_train_cv.OPOR_pred_train_dnn_cv)
df_xgb_train_cv['xgb_dif2'] = (df_xgb_train_cv.OPOR - df_xgb_train_cv.OPOR_pred_train_xgb_cv)*(df_xgb_train_cv.OPOR - df_xgb_train_cv.OPOR_pred_train_xgb_cv)

In [ ]:
# find the mean of dnn_dif2 and xgb_dif2 group by carriercode
df_xgb_train_cv.groupby('carriercode')['dnn_dif2'].mean()

In [ ]:
df_xgb_train_cv.groupby('carriercode')['xgb_dif2'].mean()

In [ ]:
df_xgb_test_cv.columns

In [ ]:
df_xgb_test_cv['dnn_dif2'] = (df_xgb_test_cv.OPOR - df_xgb_test_cv.OPOR_pred_test_dnn_cv)*(df_xgb_test_cv.OPOR - df_xgb_test_cv.OPOR_pred_test_dnn_cv)
df_xgb_test_cv['xgb_dif2'] = (df_xgb_test_cv.OPOR - df_xgb_test_cv.OPOR_pred_test_xgb_cv)*(df_xgb_test_cv.OPOR - df_xgb_test_cv.OPOR_pred_test_xgb_cv)

In [ ]:
df_xgb_test_cv.groupby('carriercode')['dnn_dif2'].mean()

In [ ]:
df_xgb_test_cv.groupby('carriercode')['xgb_dif2'].mean()

## Read in Stata Files

In [ ]:
stata_train_cv = pd.read_csv('/home/Data_Ubuntu/StataTrain_cv.csv')

In [ ]:
df_train_all = pd.merge(stata_train_cv, df_xgb_train_cv,  how='left', left_on=['carriercode','occasion'], right_on = ['carriercode','occasion'])

In [ ]:
df_train_all.shape

In [ ]:
plt.plot(df_train_all['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 7,alpha = 0.7)
plt.plot(df_train_all['OPOR_pred_train_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(df_train_all['OPOR_pred_train_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(df_train_all['pout_fe_cv'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(df_train_all['pout_xtmixed_cv'], label= "Mixed Effects Predicted OPOR", color = 'Purple', linestyle ='dashdot')

plt.xlabel('All Carriers Training Monte Carlo Observations')
plt.ylabel('OPOR')
plt.xticks([])

plt.title('All Carriers Training Monte Carlo Predictions')
plt.legend()
plt.legend(loc=4, prop={'size': 8})

plt.savefig('AllCarrier_MonteCarlo_Train_CV.pdf')

In [ ]:
american_train = df_train_all[df_train_all['carriercode'] == 6]

delta_train = df_train_all[df_train_all['carriercode'] == 11]

sw_train = df_train_all[df_train_all['carriercode'] == 24]

united_train = df_train_all[df_train_all['carriercode'] == 26]

plt.figure(figsize = (12,8))

## American 
plt.subplot(2,2,1)
plt.plot(american_train['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(american_train['OPOR_pred_train_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(american_train['OPOR_pred_train_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(american_train['pout_fe_cv'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(american_train['pout_xtmixed_cv'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('American Monte Carlo Training Observations')
plt.ylabel('OPOR')
plt.title('American Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})


## Delta
plt.subplot(2,2,2)
plt.plot(delta_train['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(delta_train['OPOR_pred_train_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(delta_train['OPOR_pred_train_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(delta_train['pout_fe_cv'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(delta_train['pout_xtmixed_cv'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Delta Monte Carlo Training Observations')
plt.ylabel('OPOR')
plt.title('Delta Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## Southwest
plt.subplot(2,2,3)
plt.plot(sw_train['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(sw_train['OPOR_pred_train_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(sw_train['OPOR_pred_train_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(sw_train['pout_fe_cv'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(sw_train['pout_xtmixed_cv'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('SouthWest Monte Carlo Training Observations')
plt.ylabel('OPOR')
plt.title('SouthWest Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## United
plt.subplot(2,2,4)
plt.plot(united_train['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(united_train['OPOR_pred_train_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(united_train['OPOR_pred_train_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(united_train['pout_fe_cv'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(united_train['pout_xtmixed_cv'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('United Monte Carlo Training Observations')
plt.ylabel('OPOR')
plt.title('United Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

plt.savefig('Big4_MonteCarlo_Train_CV.pdf')

#### Monte Carlo Testing Figures

In [ ]:
stata_test = pd.read_csv('/home/kuangwenyi/Data_Ubuntu/StataTest_cv.csv')

In [ ]:
df_test_all = pd.merge(stata_test, df_xgb_test_cv,  how='left', left_on=['carriercode','occasion'], right_on = ['carriercode','occasion'])

In [ ]:
plt.plot(df_test_all['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 4,alpha = 0.7)
plt.plot(df_test_all['OPOR_pred_test_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(df_test_all['OPOR_pred_test_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(df_test_all['pout_fe_cv_test'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(df_test_all['pout_xtmixed_cv_test'], label= "Mixed Effects Predicted OPOR", color = 'Purple', linestyle ='dashdot')

plt.xlabel('Monte Carlo Testing Observations')
plt.ylabel('OPOR')
plt.xticks([])

plt.title('All Carriers Monte Carlo Testing Predictions')
plt.legend()
plt.legend(loc=4, prop={'size': 8})

plt.savefig('AllCarrier_MonteCarlo_Test_CV.pdf')

In [ ]:
df_test_all.columns

In [ ]:
american_test = df_test_all[df_test_all['carriercode'] == 6]

delta_test = df_test_all[df_test_all['carriercode'] == 11]

sw_test = df_test_all[df_test_all['carriercode'] == 24]

united_test = df_test_all[df_test_all['carriercode'] == 26]

plt.figure(figsize = (12,8))

## American 
plt.subplot(2,2,1)
plt.plot(american_test['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(american_test['OPOR_pred_test_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(american_test['OPOR_pred_test_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(american_test['pout_fe_cv_test'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(american_test['pout_xtmixed_cv_test'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('American Monte Carlo Testing Observations')
plt.ylabel('OPOR')
plt.title('American Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})


## Delta
plt.subplot(2,2,2)
plt.plot(delta_test['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(delta_test['OPOR_pred_test_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(delta_test['OPOR_pred_test_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(delta_test['pout_fe_cv_test'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(delta_test['pout_xtmixed_cv_test'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Delta Monte Carlo Testing Observations')
plt.ylabel('OPOR')
plt.title('Delta Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## Southwest
plt.subplot(2,2,3)
plt.plot(sw_test['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(sw_test['OPOR_pred_test_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(sw_test['OPOR_pred_test_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(sw_test['pout_fe_cv_test'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(sw_test['pout_xtmixed_cv_test'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('SouthWest Monte Carlo Testing Observations')
plt.ylabel('OPOR')
plt.title('SouthWest Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## United
plt.subplot(2,2,4)
plt.plot(united_test['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(united_test['OPOR_pred_test_dnn_cv'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(united_test['OPOR_pred_test_xgb_cv'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(united_test['pout_fe_cv_test'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(united_test['pout_xtmixed_cv_test'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('United Monte Carlo Testing Observations')
plt.ylabel('OPOR')
plt.title('United Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

plt.savefig('Big4_MonteCarlo_Test_CV.pdf')

# PART 3 Predicting Future Financials (last 8 Quarters)

In [ ]:
df_raw = pd.read_csv('/home/Data_Ubuntu/E1_891.csv')

In [ ]:
df_raw = df_raw[df_raw.carriercode != 3]
df_raw = df_raw[df_raw.carriercode != 5]
df_raw = df_raw[df_raw.carriercode != 7]
df_raw = df_raw[df_raw.carriercode != 17]

In [ ]:
df_raw.carriercode.unique()
df = df_raw

In [ ]:
# convert catigorical var
cat_vars = ['carriercode', 'year', 'quarter']
# cat_vars = ['quarter']
data_cat, cat_var_list = md.cat_conv(df[cat_vars])

# define continuous variables
cont_vars = ['OPOR', 'lyield', 'lLFP', 'lfleetutil', 'lfueleff', 'lpOntime',
       'lmisbagg', 'ltotaldelay', 'lcomplaint', 'lhetero', 'lavglandfee',
       'lsparsity', 'lMKTshare', 'ltdomt_cost', 'lrevpaxenplaned', 'lempfte', 'occasion', 'carriercode']

# combine data
total_list = [df[cont_vars]]
total_list.append(data_cat)

total_df = pd.concat(total_list, axis=1)

In [ ]:
# remove the last 8 observations  for each carriercode
train_df = total_df.groupby('carriercode', group_keys=False, as_index=False).apply(lambda x: x.iloc[:-8])
y_train = np.array(train_df[['OPOR']])
X_train = np.array(train_df.drop('OPOR',axis=1))

# keep only the last 8 observations for each carriercode
test_df = total_df.groupby('carriercode', group_keys=False, as_index=False).apply(lambda x: x.iloc[-8:])
y_test = np.array(test_df[['OPOR']])
X_test = np.array(test_df.drop('OPOR',axis=1))

In [ ]:
test_df.to_csv('last160.csv', index = False)
train_df.to_csv('first705.csv', index = False)

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
# re-read in test dataframe and train dataframe
test_df = pd.read_csv('/home/last160.csv')
train_df = pd.read_csv('/home/first705.csv')

In [ ]:
print(train_df.shape)
print(test_df.shape)

## DNN 

In [ ]:
keras.utils.set_random_seed(1234) # 0.0026 loss for this random seed
# Save the number of columns in predictors: n_cols
n_cols = X_train.shape[1]

# Specify the model_dnn_test
layer = Dense(11, )

model_dnn_test = Sequential()
model_dnn_test.add(Dense(500, activation='elu', input_shape = (n_cols,), kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(450, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(400, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(600, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(350, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(300, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(250, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(200, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(100, activation='elu', kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())

model_dnn_test.add(Dense(50, activation='elu',  kernel_initializer=keras.initializers.HeNormal()))
model_dnn_test.add(BatchNormalization())
model_dnn_test.add(Dropout(0.2)) # drop out

model_dnn_test.add(Dense(1))

# Compile the model_dnn_test
model_dnn_test.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience = 2)

# Fit the model_dnn_test
model_dnn_test.fit(X_train, y_train, epochs = 100, validation_split = 0.1)
          #callbacks = [early_stopping_monitor])

##### DNN Prediction

In [ ]:
pred_train_dnn = pd.DataFrame(model_dnn_test.predict(X_train))
pred_train_dnn.columns = ['OPOR_pred_train_dnn']
# pred_test.to_csv('pred_test1.csv')

In [ ]:
mse_dnn_train = mean_squared_error(y_train, model_dnn_test.predict(X_train))
r2_dnn_train = r2_score(y_train, model_dnn_test.predict(X_train))

print("Mean Squared Error DNN Train:",mse_dnn_train)
print("R-squared Score DNN Train:", r2_dnn_train)

In [ ]:
pred_test_dnn = pd.DataFrame(model_dnn_test.predict(X_test))
pred_test_dnn.columns = ['OPOR_pred_test_dnn']
# pred_test.to_csv('pred_test1.csv')

In [ ]:
mse_dnn_test = mean_squared_error(y_test, model_dnn_test.predict(X_test))
r2_dnn_test = r2_score(y_test, model_dnn_test.predict(X_test))

print("Mean Squared Error DNN Test:",mse_dnn_test)
print("R-squared Score DNN Test:", r2_dnn_test)

In [ ]:
df_dnn_train = pd.concat((train_df[['OPOR', 'carriercode', 'occasion']], pred_train_dnn), axis =1)
#df_dnn_test.to_csv('df_dnn_test.csv')

In [ ]:
df_dnn_test = pd.concat((test_df[['OPOR', 'carriercode', 'occasion']], pred_test_dnn), axis =1)
#df_dnn_test.to_csv('df_dnn_test.csv')

## XGB

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#Creating an XGBoost regressor
model_xgb_test = xgb.XGBRegressor()

#Training the model on the training data
model_xgb_test.fit(X_train, y_train)

#Making predictions on the train data
pred_xgb_train = model_xgb_test.predict(X_train)

#Making predictions on the test set
pred_xgb_test = model_xgb_test.predict(X_test)

# Calculate the mean squared error and R-squared score for Train
mse_xgb_train = mean_squared_error(y_train, pred_xgb_train)
r2_xgb_train  = r2_score(y_train, pred_xgb_train)

# Calculate the mean squared error and R-squared score
mse_xgb_test  = mean_squared_error(y_test, pred_xgb_test)
r2_xgb_test  = r2_score(y_test, pred_xgb_test)

print("Mean Squared Error XGB Train:", mse_xgb_train)
print("R-squared Score XGB Train:", r2_xgb_train )

print("Mean Squared Error XGB Test:", mse_xgb_test )
print("R-squared Score XGB Test:", r2_xgb_test)

In [ ]:
pred_xgb_train = pd.DataFrame(model_xgb_test.predict(X_train))

pred_xgb_train.columns = ['OPOR_pred_train_xgb']

In [ ]:
pred_xgb_test = pd.DataFrame(model_xgb_test.predict(X_test))

pred_xgb_test.columns = ['OPOR_pred_test_xgb']

In [ ]:
df_xgb_train = pd.concat([df_dnn_train, pred_xgb_train], axis =1)

In [ ]:
df_xgb_test = pd.concat([df_dnn_test, pred_xgb_test], axis =1)

In [ ]:
df_xgb_train.shape

In [ ]:
df_xgb_train['dnn_dif2'] = (df_xgb_train.OPOR - df_xgb_train.OPOR_pred_train_dnn)*(df_xgb_train.OPOR - df_xgb_train.OPOR_pred_train_dnn)
df_xgb_train['xgb_dif2'] = (df_xgb_train.OPOR - df_xgb_train.OPOR_pred_train_xgb)*(df_xgb_train.OPOR - df_xgb_train.OPOR_pred_train_xgb)

In [ ]:
df_xgb_test['dnn_dif2'] = (df_xgb_test.OPOR - df_xgb_test.OPOR_pred_test_dnn)*(df_xgb_test.OPOR - df_xgb_test.OPOR_pred_test_dnn)
df_xgb_test['xgb_dif2'] = (df_xgb_test.OPOR - df_xgb_test.OPOR_pred_test_xgb)*(df_xgb_test.OPOR - df_xgb_test.OPOR_pred_test_xgb)

In [ ]:
df_xgb_train.groupby('carriercode')['dnn_dif2'].mean()

In [ ]:
df_xgb_train.groupby('carriercode')['xgb_dif2'].mean()

In [ ]:
df_xgb_test.groupby('carriercode')['dnn_dif2'].mean()

In [ ]:
df_xgb_test.groupby('carriercode')['xgb_dif2'].mean()

## Read in Training output file created by Stata

In [ ]:
stata_in = pd.read_csv('/home/Data_Ubuntu/StataOut_insample.csv')

In [ ]:
df_in_all = pd.merge(stata_in, df_xgb_train,  how='left', left_on=['carriercode','occasion'], right_on = ['carriercode','occasion'])

In [ ]:
df_in_all.shape

In [ ]:
df_in_all  = df_in_all [df_in_all ['pout_fe_in'].notna()].reset_index()

In [ ]:
df_in_all.to_csv('df_in_all.csv')

##### ALL CARRIERS IN SAMPLE

In [ ]:
plt.plot(df_in_all['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 7,alpha = 0.7)
plt.plot(df_in_all['OPOR_pred_train_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(df_in_all['OPOR_pred_train_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(df_in_all['pout_fe_in'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(df_in_all['pout_xtmixed_in'], label= "Mixed Effects Predicted OPOR", color = 'Purple', linestyle ='dashdot')

plt.xlabel('All Carriers In Sample Observations')
plt.ylabel('OPOR')
plt.xticks([])

plt.title('All Carriers In Sample Prediction')
plt.legend()
plt.legend(loc=4, prop={'size': 8})

plt.savefig('AllCarrierInSample.pdf')

##### BIG FOUR IN SAMPLE

In [ ]:
american_in = df_in_all[df_in_all['carriercode'] == 6]

delta_in = df_in_all[df_in_all['carriercode'] == 11]

sw_in = df_in_all[df_in_all['carriercode'] == 24]

united_in = df_in_all[df_in_all['carriercode'] == 26]

plt.figure(figsize = (12,8))

## American 
plt.subplot(2,2,1)
plt.plot(american_in['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(american_in['OPOR_pred_train_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(american_in['OPOR_pred_train_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(american_in['pout_fe_in'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(american_in['pout_xtmixed_in'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('American In Sample Quarters')
plt.ylabel('OPOR')
plt.title('American Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})


## Delta
plt.subplot(2,2,2)
plt.plot(delta_in['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(delta_in['OPOR_pred_train_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(delta_in['OPOR_pred_train_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(delta_in['pout_fe_in'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(delta_in['pout_xtmixed_in'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Delta In Sample Quarters')
plt.ylabel('OPOR')
plt.title('Delta Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## Southwest
plt.subplot(2,2,3)
plt.plot(sw_in['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(sw_in['OPOR_pred_train_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(sw_in['OPOR_pred_train_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(sw_in['pout_fe_in'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(sw_in['pout_xtmixed_in'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('SouthWest In Sample Quarters')
plt.ylabel('OPOR')
plt.title('SouthWest Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## United
plt.subplot(2,2,4)
plt.plot(united_in['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(united_in['OPOR_pred_train_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(united_in['OPOR_pred_train_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(united_in['pout_fe_in'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(united_in['pout_xtmixed_in'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('United In Sample Quarters')
plt.ylabel('OPOR')
plt.title('United Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

plt.savefig('Big4InSample.pdf')

## OUT OF SAMPLE Last 8 Quarters

#### Read in last 8 quarter file created by Stata

In [ ]:
stata_8 = pd.read_csv('/home/Data_Ubuntu/StataOut_8.csv')

In [ ]:
df_xgb_test.head()

In [ ]:
df_8_all = pd.merge(stata_8, df_xgb_test,  how='left', left_on=['carriercode','occasion'], right_on = ['carriercode','occasion'])

In [ ]:
df_8_all['avg_mix_xgb'] = (df_8_all['pout_xtmixed_8'] + df_8_all['OPOR_pred_test_xgb'])/2

In [ ]:
df_8_clean = df_8_all[df_8_all['OPOR'].notna()].reset_index()

#### ALL CARRIERS OUT OF SAMPLE

In [ ]:
plt.plot(df_8_clean['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 4,alpha = 0.7)
plt.plot(df_8_clean['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(df_8_clean['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(df_8_clean['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(df_8_clean['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'Purple', linestyle ='dashdot')

plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.xticks([])

plt.title('All Carriers Out of Sample Prediction')
plt.legend()
plt.legend(loc=4, prop={'size': 8})

plt.savefig('AllCarrierLast8.pdf')

#### BIG FOUR OUT OF SAMPLE

In [ ]:
american8 = df_8_all[df_8_all['carriercode'] == 6]

delta8 = df_8_all[df_8_all['carriercode'] == 11]

sw8 = df_8_all[df_8_all['carriercode'] == 24]

united8 = df_8_all[df_8_all['carriercode'] == 26]

plt.figure(figsize = (12,8))

## American 
plt.subplot(2,2,1)
plt.plot(american8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(american8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(american8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(american8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(american8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('American Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})


## Delta
plt.subplot(2,2,2)
plt.plot(delta8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(delta8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(delta8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(delta8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(delta8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('Delta Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## Southwest
plt.subplot(2,2,3)
plt.plot(sw8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(sw8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(sw8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(sw8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(sw8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('SouthWest Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## United
plt.subplot(2,2,4)
plt.plot(united8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(united8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(united8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(united8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(united8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')

plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('United Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

plt.savefig('Big4Last8Q.pdf')

In [ ]:
df_8_all['avg_dnn_xgb'] = (df_8_all['OPOR_pred_test_dnn'] + df_8_all['OPOR_pred_test_xgb'])/2

In [ ]:
american8 = df_8_all[df_8_all['carriercode'] == 6]

delta8 = df_8_all[df_8_all['carriercode'] == 11]

sw8 = df_8_all[df_8_all['carriercode'] == 24]

united8 = df_8_all[df_8_all['carriercode'] == 26]

plt.figure(figsize = (12,8))

## American 
plt.subplot(2,2,1)
plt.plot(american8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(american8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(american8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(american8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(american8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')
plt.plot(american8['avg_dnn_xgb'], label= "Average of DNN + XGB", color = 'black', linestyle = 'solid', marker = 'x')


plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('American Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})


## Delta
plt.subplot(2,2,2)
plt.plot(delta8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(delta8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(delta8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(delta8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(delta8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')
plt.plot(delta8['avg_mix_xgb'], label= "Average of Mixed Effect + XGB", color = 'black', linestyle = 'solid',marker = 'x')


plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('Delta Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## Southwest
plt.subplot(2,2,3)
plt.plot(sw8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(sw8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(sw8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(sw8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(sw8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')


plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('SouthWest Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

## United
plt.subplot(2,2,4)
plt.plot(united8['OPOR'], label= "Actual OPOR", color = 'slategrey', linewidth= 6,alpha = 0.7)
plt.plot(united8['OPOR_pred_test_dnn'], label= "DNN Predicted OPOR", color = 'blue', linestyle ='dotted')
plt.plot(united8['OPOR_pred_test_xgb'], label= "XGB Predicted OPOR", color = 'green')
plt.plot(united8['pout_fe_8'], label= "Fixed Effects Predicted OPOR", color = 'darkorange', linestyle ='dashed')
plt.plot(united8['pout_xtmixed_8'], label= "Mixed Effects Predicted OPOR", color = 'purple', linestyle = 'dashdot')


plt.xlabel('Last 8 Quarters as Out of Sample')
plt.ylabel('OPOR')
plt.title('United Airline')
plt.xticks([])

plt.legend()
plt.legend(loc=4, prop={'size': 6})

plt.savefig('Big4Last8Q_avg.pdf')